# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 30 2022 2:51PM,247676,21,629199,"NBTY Global, Inc.",Executing
1,Sep 30 2022 2:32PM,247673,10,8593052,Eywa Pharma Inc.,Released
2,Sep 30 2022 2:25PM,247672,10,SONSB0001963,"Nextsource Biotechnology, LLC",Released
3,Sep 30 2022 2:25PM,247672,10,SONSB0001964,"Nextsource Biotechnology, LLC",Released
4,Sep 30 2022 2:16PM,247671,19,ADV80006135,"AdvaGen Pharma, Ltd",Released
5,Sep 30 2022 2:16PM,247671,19,ADV80006136,"AdvaGen Pharma, Ltd",Released
6,Sep 30 2022 2:16PM,247671,19,ADV80006137,"AdvaGen Pharma, Ltd",Released
7,Sep 30 2022 2:16PM,247671,19,ADV80006138,"AdvaGen Pharma, Ltd",Released
8,Sep 30 2022 2:16PM,247671,19,ADV80006139,"AdvaGen Pharma, Ltd",Released
9,Sep 30 2022 2:16PM,247671,19,ADV80006140,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
21,247668,Released,37
22,247671,Released,6
23,247672,Released,2
24,247673,Released,1
25,247676,Executing,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247668,NaN,1.0,37.0
247671,NaN,NaN,6.0
247672,NaN,NaN,2.0
247673,NaN,NaN,1.0
247676,NaN,1.0,NaN


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247567,5.0,12.0,1.0
247601,0.0,0.0,6.0
247602,0.0,1.0,0.0
247612,0.0,9.0,6.0
247621,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247567,5,12,1
247601,0,0,6
247602,0,1,0
247612,0,9,6
247621,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247567,5,12,1
1,247601,0,0,6
2,247602,0,1,0
3,247612,0,9,6
4,247621,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247567,5,12,1
1,247601,,,6
2,247602,,1,
3,247612,,9,6
4,247621,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 30 2022 2:51PM,247676,21,"NBTY Global, Inc."
1,Sep 30 2022 2:32PM,247673,10,Eywa Pharma Inc.
2,Sep 30 2022 2:25PM,247672,10,"Nextsource Biotechnology, LLC"
4,Sep 30 2022 2:16PM,247671,19,"AdvaGen Pharma, Ltd"
10,Sep 30 2022 1:58PM,247668,10,Emerginnova
48,Sep 30 2022 1:07PM,247661,19,"NAPP Technologies, LLC"
49,Sep 30 2022 12:16PM,247655,10,Aerogen LTD.
51,Sep 30 2022 11:53AM,247653,10,ISDIN Corporation
52,Sep 30 2022 11:51AM,247645,10,"Citieffe, Inc."
60,Sep 30 2022 11:31AM,247647,10,"Nextsource Biotechnology, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 30 2022 2:51PM,247676,21,"NBTY Global, Inc.",,1,
1,Sep 30 2022 2:32PM,247673,10,Eywa Pharma Inc.,,,1
2,Sep 30 2022 2:25PM,247672,10,"Nextsource Biotechnology, LLC",,,2
3,Sep 30 2022 2:16PM,247671,19,"AdvaGen Pharma, Ltd",,,6
4,Sep 30 2022 1:58PM,247668,10,Emerginnova,,1,37
5,Sep 30 2022 1:07PM,247661,19,"NAPP Technologies, LLC",,,1
6,Sep 30 2022 12:16PM,247655,10,Aerogen LTD.,,,2
7,Sep 30 2022 11:53AM,247653,10,ISDIN Corporation,,1,
8,Sep 30 2022 11:51AM,247645,10,"Citieffe, Inc.",,1,7
9,Sep 30 2022 11:31AM,247647,10,"Nextsource Biotechnology, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 30 2022 2:51PM,247676,21,"NBTY Global, Inc.",,1,
1,Sep 30 2022 2:32PM,247673,10,Eywa Pharma Inc.,1,,
2,Sep 30 2022 2:25PM,247672,10,"Nextsource Biotechnology, LLC",2,,
3,Sep 30 2022 2:16PM,247671,19,"AdvaGen Pharma, Ltd",6,,
4,Sep 30 2022 1:58PM,247668,10,Emerginnova,37,1,
5,Sep 30 2022 1:07PM,247661,19,"NAPP Technologies, LLC",1,,
6,Sep 30 2022 12:16PM,247655,10,Aerogen LTD.,2,,
7,Sep 30 2022 11:53AM,247653,10,ISDIN Corporation,,1,
8,Sep 30 2022 11:51AM,247645,10,"Citieffe, Inc.",7,1,
9,Sep 30 2022 11:31AM,247647,10,"Nextsource Biotechnology, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 30 2022 2:51PM,247676,21,"NBTY Global, Inc.",,1,
1,Sep 30 2022 2:32PM,247673,10,Eywa Pharma Inc.,1,,
2,Sep 30 2022 2:25PM,247672,10,"Nextsource Biotechnology, LLC",2,,
3,Sep 30 2022 2:16PM,247671,19,"AdvaGen Pharma, Ltd",6,,
4,Sep 30 2022 1:58PM,247668,10,Emerginnova,37,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 30 2022 2:51PM,247676,21,"NBTY Global, Inc.",NaN,1.0,NaN
1,Sep 30 2022 2:32PM,247673,10,Eywa Pharma Inc.,1.0,NaN,NaN
2,Sep 30 2022 2:25PM,247672,10,"Nextsource Biotechnology, LLC",2.0,NaN,NaN
3,Sep 30 2022 2:16PM,247671,19,"AdvaGen Pharma, Ltd",6.0,NaN,NaN
4,Sep 30 2022 1:58PM,247668,10,Emerginnova,37.0,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 30 2022 2:51PM,247676,21,"NBTY Global, Inc.",0.0,1.0,0.0
1,Sep 30 2022 2:32PM,247673,10,Eywa Pharma Inc.,1.0,0.0,0.0
2,Sep 30 2022 2:25PM,247672,10,"Nextsource Biotechnology, LLC",2.0,0.0,0.0
3,Sep 30 2022 2:16PM,247671,19,"AdvaGen Pharma, Ltd",6.0,0.0,0.0
4,Sep 30 2022 1:58PM,247668,10,Emerginnova,37.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2228850,57.0,3.0,0.0
15,742818,9.0,23.0,8.0
19,990572,9.0,1.0,0.0
20,495256,1.0,1.0,0.0
21,247676,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2228850,57.0,3.0,0.0
1,15,742818,9.0,23.0,8.0
2,19,990572,9.0,1.0,0.0
3,20,495256,1.0,1.0,0.0
4,21,247676,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,57.0,3.0,0.0
1,15,9.0,23.0,8.0
2,19,9.0,1.0,0.0
3,20,1.0,1.0,0.0
4,21,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,57.0
1,15,Released,9.0
2,19,Released,9.0
3,20,Released,1.0
4,21,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20,21
Status,,,,,
Completed,0.0,8.0,0.0,0.0,0.0
Executing,3.0,23.0,1.0,1.0,1.0
Released,57.0,9.0,9.0,1.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20,21
0,Completed,0.0,8.0,0.0,0.0,0.0
1,Executing,3.0,23.0,1.0,1.0,1.0
2,Released,57.0,9.0,9.0,1.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20,21
0,Completed,0.0,8.0,0.0,0.0,0.0
1,Executing,3.0,23.0,1.0,1.0,1.0
2,Released,57.0,9.0,9.0,1.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()